In [26]:
%matplotlib inline
from pbg import PBG
from util import Loader
from sklearn.pipeline import Pipeline
from preprocessor import Preprocessor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# carrega matrix
s_dataset = '/exp/datasets/docs_rotulados/SyskillWebert-Parsed'
l = Loader()
d = l.from_files(s_dataset)

# Préprocessamento
cvect = CountVectorizer()
steps = [('preprocessor',Preprocessor()), ('countvectorizer',cvect)]
pipe = Pipeline(steps) 
pipe.fit(d['corpus'])
M = pipe.transform(d['corpus'])


In [27]:
from util import ConfigLabels
%reload_ext autoreload
%autoreload 2

# escolhes quantos dados rotulados
number_labelled_examples = [10,20,30,40,50] # 10,20,..,50 exemplos rotulados por classe
conf_labels = ConfigLabels(list_n_labels=number_labelled_examples)
conf_labels.fit(d['class_index']) 

In [31]:
from pbg import PBG
%reload_ext autoreload
%autoreload 2

# teste com 50 dados rotulados
n_examples_labelled = 50
#vetor y com os rótulos, apenas 50 exemplos rotulados
y = conf_labels.semi_labels[n_examples_labelled]
# numero de classes
K = len(np.unique(d['class_index'])) 

pbg = PBG(K, alpha=0.5, beta=0.001, local_max_itr=10, 
              global_max_itr=10, local_threshold = 1e-6, global_threshold = 1e-6, 
              max_time=18000, save_interval=-1, out_dir='.', out_A='A', out_B='B', calc_q=False, debug=False)
# executa pbg transdutivo
pbg.fit(M, y)


PBG(alpha=0.5, beta=0.001, calc_q=False, debug=False, global_max_itr=10,
  global_threshold=1e-06, local_max_itr=10, local_threshold=1e-06,
  max_time=18000, n_components=4, out_A='A', out_B='B', out_dir='.',
  save_interval=-1)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

y_unlabelled = conf_labels.unlabelled_idx[n_examples_labelled]
predicted_labels = pbg.transduction_[y_unlabelled]
true_labels = np.array(d['class_index'])[y_unlabelled]

cm = confusion_matrix(true_labels, predicted_labels, labels=pbg.classes_)
print(classification_report(true_labels, predicted_labels))
print("Confusion matrix")
print(cm)

             precision    recall  f1-score   support

          0       0.63      0.59      0.61        32
          1       0.98      0.84      0.90        95
          2       0.72      0.84      0.77        37
          3       0.74      1.00      0.85        25

avg / total       0.84      0.82      0.82       189

Confusion matrix
[[19  0  6  7]
 [ 8 80  6  1]
 [ 3  2 31  1]
 [ 0  0  0 25]]


In [35]:
#ABAIXO ESTÁ EXEMPLO COM O LABEL PROPAGATION

In [36]:
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import csgraph

# Learn with LabelSpreading
label_spread = label_propagation.LabelSpreading(kernel='knn', alpha=0.8)
label_spread.fit(M.toarray(), y)

/home/thiagodepaulo/anaconda3/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:288: ConvergenceWarning: max_iter=30 was reached without convergence.
  category=ConvergenceWarning


LabelSpreading(alpha=0.8, gamma=20, kernel='knn', max_iter=30, n_jobs=1,
        n_neighbors=7, tol=0.001)

In [37]:
y_unlabelled = conf_labels.unlabelled_idx[n_examples_labelled]
predicted_labels = label_spread.transduction_[y_unlabelled]
true_labels = np.array(d['class_index'])[y_unlabelled]

cm = confusion_matrix(true_labels, predicted_labels, labels=label_spread.classes_)
print(classification_report(true_labels, predicted_labels))
print("Confusion matrix")
print(cm)

             precision    recall  f1-score   support

          0       0.41      0.91      0.56        32
          1       1.00      0.52      0.68        95
          2       0.40      0.70      0.51        37
          3       1.00      0.16      0.28        25

avg / total       0.78      0.57      0.57       189

Confusion matrix
[[29  0  3  0]
 [17 49 29  0]
 [11  0 26  0]
 [14  0  7  4]]
